In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
# 경로 설정
%cd /content/drive/MyDrive/성균관대학교 과제/3학년 2학기/캡스톤설계프로젝트

/content/drive/MyDrive/24-2 Capston


### 데이터 전처리

In [ ]:
import pandas as pd

df = pd.read_csv("final_dataset_1205.csv", encoding = 'utf-8-sig')

df.head()

In [ ]:
for i, dis in enumerate(df['disease']):
  df['disease'][i]=[dis[2:-2]]

for i, dis in enumerate(df['final_symptoms']):
  df['final_symptoms'][i]=[item.strip().strip("'\"") for item in dis[2:-2].split(",")]

스트리밍 출력 내용이 길어서 마지막 5000줄이 삭제되었습니다.
See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy

  df['final_symptoms'][i]=[item.strip().strip("'\"") for item in dis[2:-2].split(",")]
<ipython-input-4-105c84c6577c>:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['final_symptoms'][i]=[item.strip().strip("'\"") for item in dis[2:-2].split(",")]
<ipython-input-4-105c84c6577c>:5: FutureWarning: ChainedAssignmentError: behaviour will change in pandas 3.0!
You are setting values through chained assignment. Currently this works in certain cases, but when using Copy-on-Write (which will become the default behaviour in pandas 3.0) this will never work to update the original DataFrame or Series, because the intermediate object on 

In [ ]:
# 데이터 유형 변환
df['cause'] = df['cause'].astype(int)
df['frequency'] = df['frequency'].astype(int)
df['duration'] = df['duration'].astype(int)
df['age'] = df['age'].astype(int)
df['social'] = df['social'].astype(int)
df['academic'] = df['academic'].astype(int)
df['occupational'] = df['occupational'].astype(int)
df['life-threatening'] = df['life-threatening'].astype(int)

In [ ]:
df.drop(columns=['Unnamed: 0','Unnamed: 0.1','Unnamed: 0.2'], inplace=True)

In [ ]:
# final_symptoms 컬럼의 빈 리스트에 '무증상' 추가
df['final_symptoms'] = df['final_symptoms'].apply(lambda x: ['무증상'] if not x else x)

###Dataload

In [ ]:
# 라이브러리 로드
import numpy as np
import pandas as pd
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
from torch import Tensor
import sklearn
import transformers
from torch.nn import BCEWithLogitsLoss
from torch.utils.data import DataLoader, Dataset, TensorDataset
from transformers import AutoModel, AutoTokenizer, AdamW, get_linear_schedule_with_warmup
from sklearn.preprocessing import LabelEncoder, MultiLabelBinarizer
from sklearn.feature_extraction.text import CountVectorizer, TfidfVectorizer
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score, confusion_matrix, classification_report
import openai
from openai import OpenAI
from torch.utils.data import TensorDataset
from transformers import BertTokenizer
from tqdm import tqdm
import pickle
import math
import copy
from torch.cuda.amp import autocast, GradScaler
#BERT 모델 import 추가 !!
from transformers import BertModel
#스케줄러 import 추가 !!
from torch.optim.lr_scheduler import ReduceLROnPlateau

In [ ]:
# 라이브러리 버전 확인
print("Numpy version:", np.__version__)
print("Pandas version:", pd.__version__)
print("PyTorch version:", torch.__version__) # BCEWithLogitsLoss도 torch 버전 확인
#print("Torch NN version:", nn.__version__)  # nn은 torch 모듈에 포함되므로 torch 버전만 확인하면 됨
#print("Torch Optim version:", optim.__version__)  # optim은 torch 모듈에 포함됨
#print("Torch Functional version:", F.__version__)  # F는 torch.nn.functional로, 별도 버전 정보는 없지만, torch 버전 참조
print("scikit-learn version:", sklearn.__version__)
print("Transformers version:", transformers.__version__)
print("OpenAI version:", openai.__version__)
#print("BertTokenizer version:", BertTokenizer.__version__)  # transformers 라이브러리의 일부분

Numpy version: 1.26.4
Pandas version: 2.2.2
PyTorch version: 2.5.1+cu121
scikit-learn version: 1.5.2
Transformers version: 4.46.3
OpenAI version: 1.54.5


In [ ]:
# gpu 디바이스 설정
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

#토크나이저 정의
tokenizer=AutoTokenizer.from_pretrained('klue/bert-base')

#max_length 정의
max_length=128

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


tokenizer_config.json:   0%|          | 0.00/289 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/425 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/248k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/495k [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/125 [00:00<?, ?B/s]

In [ ]:
#post 토큰화

all_filtered_sentences = [str(sentence) for sentence in df['filtered_sentences'].tolist()]


tokenizer = AutoTokenizer.from_pretrained('klue/bert-base')
text_inputs = tokenizer(
    all_filtered_sentences,
    padding='max_length',
    truncation=True,
    max_length=max_length,
    return_tensors="pt"
)

# post input_ids 및 attention_mask 추출
text_input_ids = text_inputs['input_ids']  # shape: (num_rows, max_length)
text_attention_mask = text_inputs['attention_mask']  # shape: (num_rows, max_length)

print(text_input_ids.shape)
print(text_attention_mask.shape)

torch.Size([5926, 128])
torch.Size([5926, 128])


In [ ]:
#evidence 관련 함수
# 토크나이징 (input_ids 및 attention_mask 생성)
def create_attention_mask(input_ids):
    return [1] * len(input_ids) + [0] * (max_length - len(input_ids))

# factor 증거(evidence) 결합
def combine_factor_inputs(evidence_input_ids_list, evidence_attention_mask_list):
    factor_input_ids, factor_attention_mask = [], []
    total_length = 0
    for evidence_input_ids, evidence_attention_mask in zip(evidence_input_ids_list, evidence_attention_mask_list):
        truncated_input_ids = evidence_input_ids[:max_length]
        truncated_attention_mask = evidence_attention_mask[:max_length]
        remaining_length = max_length - total_length
        if remaining_length <= 0:
            break
        factor_input_ids.extend(truncated_input_ids[:remaining_length])
        factor_attention_mask.extend(truncated_attention_mask[:remaining_length])
        total_length += len(truncated_input_ids)
    # 길이를 max_length로 제한
    return factor_input_ids[:max_length], factor_attention_mask[:max_length]

In [ ]:
#evidence
all_factor_input_ids = []
all_factor_attention_mask = []

for _, row in df.iterrows():  # iterrows()로 각 행 반복
    cause_evidence = row['cause_evidence']
    freq_evidence = row['freq_evidence']
    duration_evidence = row['duration_evidence']
    age_evidence = row['age_evidence']
    social_evidence = row['social_evidence']
    academic_evidence = row['academic_evidence']
    occupational_evidence = row['occupational_evidence']
    life_threatening_evidence = row['life-threatening_evidence']

    # evidence들을 각 토큰화된 input_ids로 변환
    evidence_fields = [
        cause_evidence, freq_evidence, duration_evidence, age_evidence,
        social_evidence, academic_evidence, occupational_evidence, life_threatening_evidence
    ]

    evidence_input_ids_list = [
        tokenizer.encode(evidence, add_special_tokens=True, truncation=True,
                         padding='max_length', max_length=max_length)
        for evidence in evidence_fields
    ]

    # attention_mask 생성
    evidence_attention_mask_list = [create_attention_mask(input_ids) for input_ids in evidence_input_ids_list]

    # factor 증거(evidence) 결합
    factor_input_ids, factor_attention_mask = combine_factor_inputs(
        evidence_input_ids_list, evidence_attention_mask_list
    )

    # 누적
    all_factor_input_ids.append(factor_input_ids)
    all_factor_attention_mask.append(factor_attention_mask)

# 최종 텐서로 변환
factor_input_ids = torch.tensor(all_factor_input_ids)
factor_attention_mask = torch.tensor(all_factor_attention_mask)

print(factor_input_ids.shape)  # (5926, 128)
print(factor_attention_mask.shape)  # (5926, 128)

torch.Size([5926, 128])
torch.Size([5926, 128])


In [ ]:
# 증상 데이터
'''
final_symptoms = [str(symp) for symp in df['final_symptoms'].tolist()]
symptom_inputs = tokenizer(
    final_symptoms,
    padding='max_length',
    truncation=True,
    max_length=max_length,
    return_tensors="pt"
)
symptom_input_ids = symptom_inputs['input_ids']
symptom_attention_mask = symptom_inputs['attention_mask']

print(symptom_input_ids.shape)
print(symptom_attention_mask.shape)
'''
# 고유한 48개의 증상 리스트 (모델의 타겟 클래스)
target_classes = ['무증상', '짜증이 잘 남', '두려움과 공포', '호흡곤란', '폭식', '답답함', '초조하고 안절부절 못함', '언어 능력 감소',
                  '피해망상', '불면', '성에 대한 과도한 관심', '우울함', '강박사고', '음식 섭취 제한', '자살사고', '다리 불편감', '기억력 저하',
                  '체중과 체형에 대한 집착', '무기력', '주의산만', '긴장', '감정 기복', '환각', '수면 무호흡증', '환청', '수면 과다', '충동성',
                  '피로', '메스꺼움', '어지러움', '눈물 흘림', '예민함', '극단적인 생각', '반복행동', '구토와 약물', '수면문제', '불안함',
                  '자극적인 행위에 몰입', '통제력 부족', '분노 조절 문제', '발작', '두통', '가슴 두근거림', '장시간 지속되는 이유 없는 불안감',
                  '집중력 저하', '일상생활 능력의 손상', '식욕감소', '걱정이 많아짐']

mlb_symp = MultiLabelBinarizer(classes=target_classes)  # MultiLabelBinarizer로 48개의 고유 증상 기준으로 변환
symptom_labels = mlb_symp.fit_transform(df['final_symptoms'])  # 레이블

/usr/local/lib/python3.10/dist-packages/sklearn/preprocessing/_label.py:900: UserWarning: unknown class(es) [''] will be ignored
  warnings.warn(


In [ ]:
#context categorical

# 모든 factors_tensor를 담을 리스트 생성
all_factors_tensor = []

for i, row in df.iterrows():  # 각 행을 순회
    # 각 특성별 텐서 정의
    factors = {
        'cause': row['cause'],
        'frequency': row['frequency'],
        'duration': row['duration'],
        'age': row['age'],
        'social': row['social'],
        'academic': row['academic'],
        'occupational': row['occupational'],
        'life-threatening': row['life-threatening']
    }

    # 특성 텐서들을 하나로 합침
    factors_tensor = torch.tensor([factors[key] for key in factors], dtype=torch.float32)

    # 결과를 리스트에 추가
    all_factors_tensor.append(factors_tensor)

# 리스트를 텐서로 변환
factors_tensor = torch.stack(all_factors_tensor)

# 최종 텐서 크기 확인
print(factors_tensor.shape)

torch.Size([5926, 8])


In [ ]:
# 16개의 고유 질병 클래스 정의
disease_classes = ["수면장애", "우울증", "섭식장애", "스트레스", "불안장애", "ADHD", "Non-disease", "공황장애",
                   "강박장애", "사회 불안장애", "중독(의존)", "하지 불안 증후군", "치매", "틱장애", "조현병", "양극성장애"]

# MultiLabelBinarizer로 레이블 이진화
mlb_disease = MultiLabelBinarizer(classes=disease_classes)
disease_labels = mlb_disease.fit_transform(df['disease'])  # 각 샘플에 대해 레이블 이진화
disease_labels.shape

(5926, 16)

## Modeling

### BERT CONTEXT

In [ ]:
# 데이터셋 생성
#dataset = CustomDataset(df, symptom_labels, disease_labels)
dataset = TensorDataset(factor_input_ids, factor_attention_mask, torch.tensor(symptom_labels, dtype=torch.float32), torch.tensor(disease_labels, dtype=torch.float32))


# 훈련 세트와 검증 세트 분할
train_size = int(0.9 * len(dataset))
val_size = len(dataset) - train_size
train_dataset, val_dataset = torch.utils.data.random_split(dataset, [train_size, val_size])

# DataLoader로 배치 분할
batch_size = 64
train_loader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True)
val_loader = DataLoader(val_dataset, batch_size=batch_size, shuffle=False)

In [ ]:
print(factor_input_ids.shape)
print(factor_attention_mask.shape)
print(torch.tensor(symptom_labels, dtype=torch.float32).shape)
print(torch.tensor(disease_labels, dtype=torch.float32).shape)

torch.Size([5926, 128])
torch.Size([5926, 128])
torch.Size([5926, 48])
torch.Size([5926, 16])


###클래스 별 가중치 계산

In [ ]:
# 클래스 가중치 계산
from sklearn.utils.class_weight import compute_class_weight
import torch

# 멀티라벨 데이터를 1차원으로 변환
# train_loader에서 모든 레이블 수집 (batch 차원 합침)
all_labels = torch.cat([labels for _, _, _, labels in train_loader], dim=0).cpu().numpy()  # (num_samples, num_labels)

# 각 클래스에 대해 가중치 계산
class_weights = []
for i in range(all_labels.shape[1]):  # 16개 클래스에 대해 반복
    class_weight = compute_class_weight(
        class_weight='balanced',
        classes=np.array([0, 1]),  # 클래스는 0과 1 (binary classification)
        y=all_labels[:, i]
    )
    class_weights.append(class_weight[1])  # 1 클래스의 가중치만 저장

# 텐서로 변환
class_weights = torch.tensor(class_weights).to(device)

# 결과 출력
print("Class Weights:", class_weights)

Class Weights: tensor([ 3.1408,  3.2204,  4.8045,  5.4087,  5.9388,  7.0729,  4.8837,  9.1318,
        10.2165, 12.2317, 19.6066, 22.0372, 31.0058, 55.5521, 62.0116, 86.0161],
       device='cuda:0', dtype=torch.float64)


###모델 클래스 수정

In [ ]:
class IntegratedModelWithBERT(nn.Module):
    def __init__(self, bert_model_name, embedding_dim, symptom_dim, num_labels, filter_num=64, filter_sizes=[1, 3, 5], max_pooling_k=1, dropout=0.2):
        super().__init__()
        self.embedding_dim = embedding_dim
        self.symptom_dim = symptom_dim
        self.num_labels = num_labels

        # BERT 모델 초기화
        self.bert = BertModel.from_pretrained(bert_model_name)
        self.bert_hidden_size = self.bert.config.hidden_size  # BERT의 출력 크기

        # CNN Layers
        self.filter_num = filter_num
        self.filter_sizes = filter_sizes
        self.hidden_size = len(filter_sizes) * filter_num
        self.convs = nn.ModuleList([nn.Conv1d(2, filter_num, size) for size in filter_sizes])  # input_channels=2

        self.max_pooling_k = max_pooling_k
        self.dropout = nn.Dropout(dropout)
        self.fc = nn.Linear(self.hidden_size + self.bert_hidden_size, num_labels)  # BERT 출력과 CNN 출력 결합

    def forward(self, factor_input_ids, factor_attention_mask, symptoms):
        # BERT 출력 계산
        bert_outputs = self.bert(input_ids=factor_input_ids, attention_mask=factor_attention_mask)
        bert_pooled_output = bert_outputs.pooler_output  # (batch_size, bert_hidden_size)

        # 차원 확장: factor_input_ids에 마지막 차원 추가
        factor_input_ids = factor_input_ids.unsqueeze(-1).float()  # (batch_size, seq_len, 1)

        # Pooling으로 차원 축소: seq_len → symptom_dim
        factors_reduced = F.adaptive_avg_pool1d(factor_input_ids.transpose(1, 2), output_size=self.symptom_dim).transpose(1, 2)  # (batch_size, symptom_dim, 1)=(64,48,1)

        # 증상 데이터 차원 맞춤
        input_seqs = torch.cat([symptoms, factors_reduced], dim=2)  # (batch_size, symptom_dim, 2)

        # Conv1d 입력 차원 변환
        input_seqs = input_seqs.transpose(1, 2)  # (batch_size, 2, symptom_dim)

        # CNN 처리
        x = [F.relu(conv(input_seqs)) for conv in self.convs]
        x = [self.kmax_pooling(item, self.max_pooling_k).mean(2) for item in x]
        x = torch.cat(x, 1)  # (batch_size, hidden_size)
        x = self.dropout(x)

        # BERT 출력과 CNN 출력을 결합
        combined = torch.cat([bert_pooled_output, x], dim=1)  # (batch_size, hidden_size + bert_hidden_size)

        # Fully Connected Layer for prediction
        logits = self.fc(combined)  # (batch_size, num_labels)
        return logits

    def kmax_pooling(self, x, k):
        if x.dim() < 3:
            raise ValueError(f"Input to kmax_pooling must have at least 3 dimensions, but got {x.shape}")
        return x.sort(dim=2, descending=True)[0][:, :, :k]


In [ ]:
from torch.optim.lr_scheduler import ReduceLROnPlateau

# 모델 초기화
embedding_dim = 128
symptom_dim = 48
num_labels = 16

# 모델 초기화
BERT_context = IntegratedModelWithBERT(
    bert_model_name="klue/bert-base",
    embedding_dim=embedding_dim,
    symptom_dim=symptom_dim,
    num_labels=num_labels,
    filter_num=128,
    filter_sizes=[1, 3, 5],
    max_pooling_k=1,
    dropout=0.2
)

# 손실 함수 및 옵티마이저
criterion = nn.BCEWithLogitsLoss(pos_weight=class_weights).to(device)  # 클래스 가중치 반영
optimizer = optim.AdamW(BERT_context.parameters(), lr=5e-5)

# 스케줄러 설정
scheduler = ReduceLROnPlateau(optimizer, mode='min', patience=2, factor=0.5)

BERT_context.to(device)

# 학습 루프
num_epochs = 10
for epoch in range(num_epochs):
    BERT_context.train()
    total_loss = 0
    for batch in train_loader:
        factor_input_ids, factor_attention_mask, symptoms, labels = [
            item.to(device) for item in batch
        ]

        optimizer.zero_grad()
        logits = BERT_context(factor_input_ids, factor_attention_mask, symptoms.unsqueeze(-1))
        loss = criterion(logits, labels)
        loss.backward()
        optimizer.step()
        total_loss += loss.item()

    avg_train_loss = total_loss / len(train_loader)
    print(f"Epoch {epoch + 1}, Train Loss: {avg_train_loss:.4f}")

    BERT_context.eval()
    total_val_loss = 0

    with torch.no_grad():
        for batch in val_loader:  # 검증 데이터 로더 사용
            factor_input_ids, factor_attention_mask, symptoms, labels = [
                item.to(device) for item in batch
            ]

            logits = BERT_context(factor_input_ids, factor_attention_mask, symptoms.unsqueeze(-1))
            loss = criterion(logits, labels)
            total_val_loss += loss.item()

    avg_val_loss = total_val_loss / len(val_loader)
    print(f"Epoch {epoch + 1}/{num_epochs}, Validation Loss: {avg_val_loss:.4f}")

    scheduler.step(avg_val_loss)

model.safetensors:   0%|          | 0.00/445M [00:00<?, ?B/s]

Epoch 1, Train Loss: 280.2119
Epoch 1/10, Validation Loss: 94.0857
Epoch 2, Train Loss: 251.3250
Epoch 2/10, Validation Loss: 98.4286
Epoch 3, Train Loss: 251.7747
Epoch 3/10, Validation Loss: 82.4083
Epoch 4, Train Loss: 246.2454
Epoch 4/10, Validation Loss: 85.5822
Epoch 5, Train Loss: 254.2032
Epoch 5/10, Validation Loss: 78.4504
Epoch 6, Train Loss: 228.3424
Epoch 6/10, Validation Loss: 74.2807
Epoch 7, Train Loss: 227.8277
Epoch 7/10, Validation Loss: 80.6254
Epoch 8, Train Loss: 224.9437
Epoch 8/10, Validation Loss: 76.8191
Epoch 9, Train Loss: 219.1835
Epoch 9/10, Validation Loss: 81.1332
Epoch 10, Train Loss: 222.3830
Epoch 10/10, Validation Loss: 65.4539


In [ ]:
# 각 타겟 threshold 출력
probabilities = torch.sigmoid(logits)
print(probabilities)
print(probabilities.mean())

tensor([[0.0000e+00, 6.8105e-27, 1.0000e+00, 0.0000e+00, 0.0000e+00, 0.0000e+00,
         0.0000e+00, 6.7293e-09, 1.1949e-36, 0.0000e+00, 0.0000e+00, 0.0000e+00,
         0.0000e+00, 0.0000e+00, 0.0000e+00, 0.0000e+00],
        [0.0000e+00, 0.0000e+00, 0.0000e+00, 5.0594e-26, 6.8514e-14, 9.9352e-01,
         0.0000e+00, 4.7911e-09, 1.0000e+00, 0.0000e+00, 0.0000e+00, 0.0000e+00,
         0.0000e+00, 8.9239e-38, 1.3111e-05, 9.2030e-26],
        [2.0485e-23, 0.0000e+00, 9.4767e-31, 4.2106e-26, 1.5810e-33, 0.0000e+00,
         2.4617e-30, 0.0000e+00, 8.3793e-22, 0.0000e+00, 0.0000e+00, 6.0024e-39,
         0.0000e+00, 0.0000e+00, 0.0000e+00, 3.9661e-29],
        [0.0000e+00, 0.0000e+00, 0.0000e+00, 7.6766e-03, 0.0000e+00, 0.0000e+00,
         0.0000e+00, 1.0000e+00, 0.0000e+00, 0.0000e+00, 0.0000e+00, 0.0000e+00,
         0.0000e+00, 0.0000e+00, 0.0000e+00, 0.0000e+00],
        [0.0000e+00, 0.0000e+00, 0.0000e+00, 1.3105e-37, 0.0000e+00, 1.0000e+00,
         0.0000e+00, 9.7905e-06, 0.0000

In [ ]:
# 모델 검증
average_threshold = probabilities.mean()  # 모델 출력의 평균 확률값을 기준으로 설정

BERT_context.eval()  # 모델 평가 모드
predictions, true_labels = [], []  # 예측값과 실제값 저장 리스트

with torch.no_grad():  # 그래디언트 계산 비활성화
    for factor_input_ids, factor_attention_mask, symptoms, labels in val_loader:
        # 입력 데이터를 GPU로 이동
        factor_input_ids = factor_input_ids.to(device)
        factor_attention_mask = factor_attention_mask.to(device)
        symptoms = symptoms.to(device)
        labels = labels.to(device)

        # 모델 예측
        logits = BERT_context(factor_input_ids, factor_attention_mask, symptoms.unsqueeze(-1))
        probabilities = torch.sigmoid(logits)  # 로짓 값을 확률로 변환

        # 임계값을 기준으로 이진화
        predicted_labels = (probabilities > average_threshold).int()
        predictions.append(predicted_labels)
        true_labels.append(labels)

# 평가 지표 계산
predictions = torch.cat(predictions).cpu()  # GPU에서 CPU로 이동
true_labels = torch.cat(true_labels).cpu()  # GPU에서 CPU로 이동

precision = precision_score(true_labels, predictions, average='macro', zero_division=1)
recall = recall_score(true_labels, predictions, average='macro', zero_division=1)
f1 = f1_score(true_labels, predictions, average='macro', zero_division=1)

# 결과 출력
print(f"Precision: {precision:.4f}, Recall: {recall:.4f}, F1 Score: {f1:.4f}")

#이전성능: Precision: 0.1987, Recall: 0.0673, F1 Score: 0.0445

Precision: 0.0519, Recall: 0.0767, F1 Score: 0.0434


In [ ]:
# 저장 파일 경로
pkl_save_path = "model/bert_context.pkl"
pth_save_path = "model/bert_context.pth"

# 전체 모델 저장 (pkl 형식)
torch.save(BERT_context, pkl_save_path)
print(f"Entire model saved to {pkl_save_path}!!!")

# 상태 사전(state_dict) 저장 (pth 형식)
torch.save(BERT_context.state_dict(), pth_save_path)
print(f"Model state_dict saved to {pth_save_path}!!!")

Entire model saved to bert_context_최종.pkl!!!
Model state_dict saved to bert_context_최종.pth!!!
